In [1]:
import LotData.LotData as ld
import LotData.ExtractorsAndTables as ent
import utility.LoggingUtility as lut
import utility.webscrapingUtil as wut
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
import CW_Scraper
import sqlalchemy
import time
import numpy as np
from sqlalchemy.orm import sessionmaker
from sqlalchemy.sql import text
from Runnables.RunningSettings import wanted_categories
import database.DatabaseManager as dbm

In [2]:
session,engine = dbm.getSessionEngine()

In [ ]:
err_lids = []
for (cat_int,cat_name) in wanted_categories.items():
    items = 0
    wttches_magov = CW_Scraper.MagazineOverview(cat_int)
    wttches_magov.set_active_nr_pages()
    for i,page_i in enumerate(wttches_magov):
        times = []
        for LID in page_i:
            t1 = time.time()
            meta_data = ent.MetadataExtractor(LID,lut.getTimeStamp(),cat_int,cat_name)

            l_data = ld.LotData(meta_data)
            for record_key in l_data.keys():
                table = wut.recordIntoTabe(record_key)
                query = f"SELECT EXISTS (SELECT 1 FROM {table} WHERE lid = :lid)"

                result = session.execute(text(query), {'lid': LID})
                exists = result.scalar()  # or fetch the result as required
                if(not exists):
                    try:

                        df = l_data[record_key]
                        if(record_key == "meta_record"):
                            df["status"] = "idle"
                        df.to_sql(record_key.replace("_record",""), con=engine, if_exists='append',index=False)
                    except Exception as e:
                        err_lids += [(LID,e)]

            times += [time.time() - t1]
            time.sleep(np.random.uniform(0,0.01))
        if(np.mean(times)>2):
            print(f"Page number: {i} out of {len(wttches_magov)} of category {cat_name} \t Average time lot: {np.mean(times)} with variance {np.var(times)}")

    print(f"Finished category: {cat_name}")


non_expected_errors = [err for err in err_lids if len(err) > 1 and (type(err[1]) != sqlalchemy.exc.IntegrityError) and (type(err[1]) != KeyError)]

Page number: 24 out of 70 of category diamonds 	 Average time lot: 2.4190571208794913 with variance 7.791613982515156
Page number: 25 out of 70 of category diamonds 	 Average time lot: 2.6890210111935935 with variance 8.19332587041985
Page number: 49 out of 70 of category diamonds 	 Average time lot: 2.2305896282196045 with variance 7.176644940921889
Page number: 50 out of 70 of category diamonds 	 Average time lot: 2.1231571634610495 with variance 6.72681136461362
Page number: 56 out of 70 of category diamonds 	 Average time lot: 2.106343686580658 with variance 6.366276507803778
Page number: 61 out of 70 of category diamonds 	 Average time lot: 2.356283515691757 with variance 7.1814635307744865
Page number: 62 out of 70 of category diamonds 	 Average time lot: 2.698876202106476 with variance 7.263774187422789
Page number: 69 out of 70 of category diamonds 	 Average time lot: 2.715789645910263 with variance 7.0911813615769566
Finished category: diamonds
Page number: 0 out of 215 of cat